In [ ]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
from numpy import pi

#Initialize Freeze Core Transformer by removing unnecessary orbital
freezeCoreTransfomer = FreezeCoreTransformer(remove_orbitals=[3,4])
#qmolecule = freezeCoreTransfomer.transform(qmolecule)

#Do the Hamiltonian Two-Body integral and store the result in main_op
es_problem = ElectronicStructureProblem(driver,q_molecule_transformers=[freezeCoreTransfomer])
second_q_ops = es_problem.second_q_ops()
main_op = second_q_ops[0]


#Select Map Type 
map_type = 'ParityMapper'

if map_type== 'ParityMapper':
    mapper = ParityMapper()
elif map_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif map_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

#Calculate the final fermionic operator and store the result in qubit_op
converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction= [1, 1])
num_particles = (es_problem.molecule_data_transformed.num_alpha,
             es_problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * es_problem.molecule_data_transformed.num_molecular_orbitals
qubit_op = converter.convert(main_op, num_particles=num_particles)

#Initialize VQE Parameters

initial_point = [0.01] * ansatz.num_parameters
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx' , entanglement='linear', reps=1, skip_final_rotation_layer=False)

ansatz.compose(init_state, front=True, inplace=True)

#ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)

optimizer = COBYLA(maxiter=20000)
backend = Aer.get_backend('statevector_simulator')
                          
#Initialize VQE Algorithm
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []
                          
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)
# WRITE YOUR CODE BETWEEN THESE LINES - END